# Web Scraping with LLM Data Extraction


## Install dependencies

In [1]:
!pip install -q BeautifulSoup4 requests

In [ ]:
!pip install -q openai

In [ ]:
!pip install -q selenium

## Configure environment variables

In [4]:
# If opening this notebook in Google Colab, add OPENAI_API_KEY to Secrets
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Define JSON Schema

In [5]:
json_schema = """
{
    "type": "object",
    "properties": {
        "bankDomain": {
            "type": "string",
            "description": "The domain of the bank to which these account types belong."
        },
        "checkingAccounts": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the checking account, e.g. Interest Checking, Senior Advantage Checking, etc."
                    },
                    "interestRate": {
                        "type": "number",
                        "description": "The interest rate of the checking account, if any."
                    },
                    "annualPercentageYield": {
                        "type": "number",
                        "description": "The annual percentage yield (APY) of the checking account."
                    },
                    "minimumBalanceToObtainAPY": {
                        "type": "number",
                        "description": "The minimum balance to obtain the annual percentage yield (APY)."
                    },
                    "minimumBalanceToOpen": {
                        "type": "number",
                        "description": "The minimum balance to open the checking account."
                    },
                    "minimumDailyBalance": {
                        "type": "number",
                        "description": "The minimum daily balance of the checking account to obtain APY or avoid fees."
                    },
                    "dividendRate": {
                        "type": "number",
                        "description": "The dividend rate of the checking account, if any."
                    },
                    "dividendFrequency": {
                        "type": "string",
                        "description": "The frequency at which dividends are paid, if at all, e.g. monthly, quarterly, annually, etc."
                    }
                },
                "required": [
                    "name",
                    "annualPercentageYield"
                ]
            }
        },
        "savingsAccounts": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the savings account, e.g. Partnership Savings, Statement Savings, etc."
                    },
                    "interestRate": {
                        "type": "number",
                        "description": "The interest rate of the savings account, if any."
                    },
                    "annualPercentageYield": {
                        "type": "number",
                        "description": "The annual percentage yield (APY) of the savings account."
                    },
                    "minimumBalanceToObtainAPY": {
                        "type": "number",
                        "description": "The minimum balance to obtain the annual percentage yield (APY)."
                    },
                    "minimumBalanceToOpen": {
                        "type": "number",
                        "description": "The minimum balance to open the savings account."
                    },
                    "minimumDailyBalance": {
                        "type": "number",
                        "description": "The minimum daily balance of the savings account to obtain APY or avoid fees."
                    },
                    "dividendRate": {
                        "type": "number",
                        "description": "The dividend rate of the checking account, if any."
                    },
                    "dividendFrequency": {
                        "type": "string",
                        "description": "The frequency at which dividends are paid, if at all, e.g. monthly, quarterly, annually, etc."
                    },
                    "required": [
                        "name",
                        "annualPercentageYield"
                    ]
                }
            },
            "moneyMarketAccounts": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name or tier of money market account, e.g. Tier 1 ($0.01 - $9,999.99), etc."
                        },
                        "interestRate": {
                            "type": "number",
                            "description": "The interest rate of the savings account, if any."
                        },
                        "annualPercentageYield": {
                            "type": "number",
                            "description": "The annual percentage yield (APY) of the savings account."
                        },
                        "minimumBalanceToObtainAPY": {
                            "type": "number",
                            "description": "The minimum balance to obtain the annual percentage yield (APY)."
                        },
                        "dividendRate": {
                            "type": "number",
                            "description": "The dividend rate of the checking account, if any."
                        },
                        "dividendFrequency": {
                            "type": "string",
                            "description": "The frequency at which dividends are paid, if at all, e.g. monthly, quarterly, annually, etc."
                        },
                        "minimumBalanceToOpen": {
                            "type": "number",
                            "description": "The minimum balance to open the savings account."
                        },
                        "minimumDailyBalance": {
                            "type": "number",
                            "description": "The minimum daily balance of the savings account to obtain APY or avoid fees."
                        }
                    },
                    "required": [
                        "name",
                        "annualPercentageYield"
                    ]
                }
            },
            "certificatesOfDeposit": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "term": {
                            "type": "string",
                            "description": "The term of the certificate of deposit, e.g. 6 months, 12 months, etc."
                        },
                        "interestRate": {
                            "type": "number",
                            "description": "The interest rate of the certificate of deposit, if any."
                        },
                        "annualPercentageYield": {
                            "type": "number",
                            "description": "The annual percentage yield (APY) of the certificate of deposit."
                        },
                        "minimumBalanceToObtainAPY": {
                            "type": "number",
                            "description": "The minimum balance to obtain the annual percentage yield (APY)."
                        },
                        "minimumBalanceToOpen": {
                            "type": "number",
                            "description": "The minimum balance to open the certificate of deposit."
                        },
                        "minimumDailyBalance": {
                            "type": "number",
                            "description": "The minimum daily balance of the certificate of deposit to obtain APY or avoid fees."
                        }
                    },
                    "required": [
                        "term",
                        "annualPercentageYield"
                    ]
                }
            },
            "individualRetirementAccounts": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "term": {
                            "type": "string",
                            "description": "The term of the individual retirement account, e.g. 7 months, 12 months, etc."
                        },
                        "interestRate": {
                            "type": "number",
                            "description": "The interest rate of the individual retirement account, if any."
                        },
                        "annualPercentageYield": {
                            "type": "number",
                            "description": "The annual percentage yield (APY) of the individual retirement account."
                        },
                        "minimumBalanceToObtainAPY": {
                            "type": "number",
                            "description": "The minimum balance to obtain the annual percentage yield (APY)."
                        },
                        "minimumBalanceToOpen": {
                            "type": "number",
                            "description": "The minimum balance to open the individual retirement account."
                        },
                        "minimumDailyBalance": {
                            "type": "number",
                            "description": "The minimum daily balance of the individual retirement account to obtain APY or avoid fees."
                        }
                    },
                    "required": [
                        "term",
                        "annualPercentageYield"
                    ]
                }
            },
            "loans": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the loan, e.g. Auto Loan, Student Loan, 30-Year Fixed Rate Mortgage, etc."
                        },
                        "term": {
                            "anyOf": [
                                {
                                    "type": "integer"
                                },
                                {
                                    "type": "string"
                                }
                            ],
                            "description": "The term of the loan, e.g. 1-3 years, 7 years, etc."
                        },
                        "annualPercentageRate": {
                            "type": "number",
                            "description": "The Annual Percentage Rate (APR) of the loan. APR is the interest rate plus additional fees charged by the lender."
                        },
                        "minimumPayment": {
                            "type": "number",
                            "description": "The required minimum monthly payment for the loan."
                        },
                        "maximumLoanAmount": {
                            "type": "number",
                            "description": "The maximum loan amount that can be borrowed as a percentage of the collateral value."
                        },
                        "paymentPer1000Dollars": {
                            "type": "number",
                            "description": "The amount the borrower would pay per month for every $1,000 borrowed."
                        }
                    },
                    "required": [
                        "name",
                        "annualPercentageRate"
                    ]
                }
            },
            "creditCards": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the credit card, if applicable, e.g. Visa Platinum, Mastercard Gold, etc."
                        },
                        "annualPercentageRate": {
                            "type": "number",
                            "description": "The Annual Percentage Rate (APR) of the loan. APR is the interest rate plus additional fees charged by the lender."
                        },
                        "annualFee": {
                            "type": "number",
                            "description": "The annual fee charged by the credit card provider."
                        },
                        "doesEarnRewards": {
                            "type": "boolean",
                            "description": "Indicates whether the credit card earns rewards or not."
                        }
                    },
                    "required": [
                        "annualPercentageRate",
                        "annualFee"
                    ]
                }
            },
            "fees": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the fee, e.g. Overdraft Fee, Wire Transfer Fee, etc."
                        },
                        "feeAmount": {
                            "type": "number",
                            "description": "The fee amount."
                        },
                        "feeUnit": {
                            "type": "string",
                            "description": "The unit of the fee amount, e.g. 'per hour', 'per month', 'per ten', '%', etc."
                        },
                        "oneTime": {
                            "type": "boolean",
                            "description": "Indicates if the fee is a one-time fee."
                        },
                        "recurringInterval": {
                            "type": "string",
                            "description": "The interval for recurring fees, e.g. 'monthly', 'annually', etc. (if applicable).",
                            "enum": ["daily", "weekly", "monthly", "annually"]
                        }
                    },
                    "required": [
                        "name",
                        "feeAmount",
                        "feeUnit"
                    ],
                    "anyOf": [
                        {
                            "required": ["oneTime"]
                        },
                        {
                            "required": ["recurringInterval"]
                        }
                    ]
                }
            }
        }
    }
}
"""

## Process URLs and extract tables in JSON format

In [6]:
import time
import random
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
from typing import List, Dict, Optional, Tuple
import csv
import io
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
import os


def fetch_tables(url: str, session: requests.Session, headers: Dict[str, str]) -> Tuple[str, Optional[List[Tag]]]:
    try:
        response = session.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            tables = soup.find_all('table')
            return url, tables
        else:
            # Fall back to Selenium if requests fails
            # Useful if websites load content dynamically with JavaScript.
            chrome_options = Options()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-dev-shm-usage')
            driver = webdriver.Chrome(service=Service(), options=chrome_options)

            driver.get(url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, 'table'))
            )
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            tables = soup.find_all('table')
            driver.quit()
            return url, tables
    except requests.RequestException as e:
        print(f"Failed to retrieve {url} with requests: {e}")
        return url, None
    except Exception as e:
        print(f"An error occurred with {url} using Selenium: {e}")
        return url, None

# Given a list of URLs, return a dictionary where each key is a URL and each value
# is a list of Tag objects representing the HTML table(s) for the given URL
def extract_all_tables_from_html(urls: List[str], max_workers=10) -> Dict[str, Optional[List[Tag]]]:
    result: Dict[str, Optional[List[Tag]]] = {}
    unsuccessful_urls: List[str] = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Connection': 'keep-alive',
        'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1'
    }

    session = requests.Session()

    with ThreadPoolExecutor(max_workers) as executor:
        future_to_url = {executor.submit(fetch_tables, url, session, headers): url for url in urls}

        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                url, tables = future.result()
                result[url] = tables
            except Exception as e:
                print(f"An error occurred with {url}: {e}")
                result[url] = None
                unsuccessful_urls.append(url)

    return result, unsuccessful_urls


# Convert a table Tag object to CSV string
def table_to_csv(table: Tag) -> str:
    output = io.StringIO()
    writer = csv.writer(output)

    for row in table.find_all('tr'):
        cols = row.find_all(['td', 'th'])
        writer.writerow([col.get_text(strip=True) for col in cols])

    return output.getvalue()

# Convert a dictionary of Tag objects to a dictionary of CSV strings
def convert_tables_to_csv(results: Dict[str, Optional[List[Tag]]]) -> Dict[str, List[str]]:
    csv_results: Dict[str, List[str]] = {}

    for url, tables in results.items():
        if tables is None:
            csv_results[url] = []
            continue

        csv_tables = []
        for table in tables:
            csv_content = table_to_csv(table)
            csv_tables.append(csv_content)

        csv_results[url] = csv_tables

    return csv_results

def chunk_data(data: str, chunk_size: int) -> List[str]:
    chunks = []
    for i in range(0, len(data), chunk_size):
        chunks.append(data[i:i + chunk_size])
    return chunks

def extract_with_llm(chunk, schema):
    prompt = f"""
Extract the banking rate data from the following text and format it according to the JSON schema provided.

Special instructions:
- If a propety or object does not exist, do not include it in the output.
- If dividend rate is given, do not include interest rate.
- Do not convert percentage to decimal. I.e. if the rate is 0.55%, return 0.55 not 0.0055

Text:
{chunk}

JSON Schema:
{schema}

Extracted Data:
"""
    client = OpenAI(
        api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
    )

    response=client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": prompt}
            ]
        )
    return response.choices[0].message.content

# Given a dictionary of tables in CSV format and a JSON schema, chunck tables,
# send them to an LLM for processing into JSON, and return the extracted JSON data
# and the total processing time per table.
def process_and_extract_tables(csv_tables_dict: Dict[str, List[str]], schema: str, chunk_size: int) -> Tuple[Dict[str, List[dict]], List[str]]:
    extracted_data: Dict[str, List[dict]] = {}
    unsuccessful_urls: List[str] = []

    for url, csv_tables in csv_tables_dict.items():
        if not csv_tables:
            unsuccessful_urls.append(url)
            continue

        start_time = time.time()
        combined_csv = ''.join(csv_tables)  # Combine all CSV tables for the URL
        chunks = chunk_data(combined_csv, chunk_size)
        extracted_chunks = []

        for chunk in chunks:
            extracted_data_chunk = extract_with_llm(chunk, schema)
            extracted_chunks.append(extracted_data_chunk)

        end_time = time.time()
        processing_time = end_time - start_time

        extracted_data[url] = extracted_chunks

        # Print the processing time for each URL
        print(f"Processing time for {url}: {processing_time:.2f} seconds")

    return extracted_data, unsuccessful_urls

In [7]:
# Print the CSV content for each URL
def print_csv_tables(csv_tables_dict):
    for url, csv_tables in csv_tables_dict.items():
        print(f"CSV Tables from {url}:")
        for i, csv_table in enumerate(csv_tables):
            print(f"Table {i+1}:")
            print(csv_table)
            print()  # Print a newline for better readability

In [8]:
urls = [
    'https://www.simplicity.coop/rates',
    'https://www.bankofdeerfield.bank/resources/deposit-rates',
    'https://www.salemcoop.com/rates-fees/deposit-interest-rates/',
    'https://www.bankwithpremier.com/Current%20Deposit%20Interest%20Rates',
    'https://www.shrewsburycu.com/home/member-services/rates',
]

tables_dict, unsuccessful_urls = extract_all_tables_from_html(urls, max_workers=10)
csv_tables_dict = convert_tables_to_csv(tables_dict)
chunk_size = 1000  # Limit in bytes
start_time = time.time()
extracted_data, extraction_unsuccessful_urls = process_and_extract_tables(csv_tables_dict, json_schema, chunk_size)
print(f"Total processing time: {time.time() - start_time:.2f} seconds")

# Combine unsuccessful URLs from both extraction stages
all_unsuccessful_urls = set(unsuccessful_urls).union(set(extraction_unsuccessful_urls))

Failed to retrieve https://www.bankofdeerfield.bank/resources/deposit-rates with requests: HTTPSConnectionPool(host='www.bankofdeerfield.bank', port=443): Max retries exceeded with url: /resources/deposit-rates (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7c080a68d1e0>, 'Connection to www.bankofdeerfield.bank timed out. (connect timeout=None)'))
Processing time for https://www.bankwithpremier.com/Current%20Deposit%20Interest%20Rates: 50.65 seconds
Processing time for https://www.salemcoop.com/rates-fees/deposit-interest-rates/: 23.23 seconds
Processing time for https://www.simplicity.coop/rates: 19.09 seconds
Processing time for https://www.shrewsburycu.com/home/member-services/rates: 1.37 seconds
Total processing time: 94.34 seconds


## Display Results

In [ ]:
# print_csv_tables(csv_tables_dict)

In [ ]:
# Printing the extracted data and processing time for each URL
for url, data in extracted_data.items():
    print(f"Extracted Data from {url}:")
    for i, chunk_data in enumerate(data):
        print(f"Chunk {i+1}:")
        print(chunk_data)
        print()  # Print a newline for better readability

# Printing the URLs that were unsuccessful
print("Unsuccessful URLs:")
for url in all_unsuccessful_urls:
    print(url)

## Handle rates across multiple pages

For banks that have rates spread across multiple web pages, we will process each page individually into a list of JSON objects, the merge those objects (avoiding duplication) into a single object for that bank.

In [ ]:
import json
from collections import defaultdict
from typing import Dict, List

def merge_json_objects_list(extracted_data: Dict[str, List[dict]]):
    if not extracted_data:
        return []

    # Flatten the list of JSON objects and group by bankName
    bank_groups = defaultdict(list)
    for url, json_objects in extracted_data.items():
        for obj in json_objects:
            bank_name = obj['bankName']
            bank_groups[bank_name].append(obj)

    # Define the sections to be merged and their unique keys
    sections = {
        "checkingAccounts": "name",
        "savingsAccounts": "name",
        "moneyMarketAccounts": "name",
        "certificatesOfDeposit": "term",
        "individualRetirementAccounts": "term",
        "loans": "name",
        "creditCards": "name",
        "fees": "name"
    }

    # Function to merge lists without duplicates
    def merge_lists(list1, list2, unique_key):
        merged_list = list1[:]
        existing_keys = {item[unique_key] for item in list1}
        for item in list2:
            if item[unique_key] not in existing_keys:
                merged_list.append(item)
                existing_keys.add(item[unique_key])
        return merged_list

    merged_objects = []

    # Merge JSON objects for each bank
    for bank_name, objects in bank_groups.items():
        merged_obj = objects[0]
        for obj in objects[1:]:
            for section, unique_key in sections.items():
                list1 = merged_obj.get(section, [])
                list2 = obj.get(section, [])
                merged_obj[section] = merge_lists(list1, list2, unique_key)
        merged_objects.append(merged_obj)

    return merged_objects


In [ ]:
multipage_urls = [
    'https://www.parkbank.bank/Pages/savings-cd.html',
    'https://www.parkbank.bank/Pages/p-checking.html',
    'https://www.parkbank.bank/Pages/rewards-checking.html',
    # 'https://www.bluffviewbank.com/personal/savings/cd-rates/',
    # 'https://www.bluffviewbank.com/personal/checking/checking-and-savings-rates/',
    # 'https://www.dcu.org/bank/certificates/regular-certificates.html',
    # 'https://www.dcu.org/bank/savings.html',
    # 'https://www.dcu.org/bank/checking.html',
    # 'https://www.dcu.org/bank/retirement/ira-savings.html',
    # 'https://www.firstiowa.bank/personal/cd-and-ira-rates',
    # 'https://www.firstiowa.bank/connect/rates',
    # 'https://www.firstiowa.bank/personal/checking',
    # 'https://www.huntington.com/Personal/savings-cds-overview/certificates-of-deposit',
    # 'https://www.huntington.com/Personal/savings-cds-overview/relationship-money-market-account',
    # 'https://www.huntington.com/Personal/checking/perks',
    # 'https://www.huntington.com/Personal/savings-cds-overview/money-market-ira',
    # 'https://www.huntington.com/Personal/savings-cds-overview/premier-savings-account',
    # 'https://verveacu.com/personal/product/savings/certificates/share-certificates/',
    # 'https://verveacu.com/personal/product/savings/money-market/',
    # 'https://verveacu.com/personal/product/checking/kickback-checking/',
    # 'https://verveacu.com/personal/product/savings/kickback-savings/',
    # 'https://verveacu.com/personal/product/savings/money-market/',
]

tables_dict, unsuccessful_urls = extract_all_tables_from_html(multipage_urls, max_workers=10)
csv_tables_dict = convert_tables_to_csv(tables_dict)
chunk_size = 1000  # Limit in bytes
start_time = time.time()
extracted_data, extraction_unsuccessful_urls = process_and_extract_tables(csv_tables_dict, json_schema, chunk_size)
print(f"Total processing time: {time.time() - start_time:.2f} seconds")

# Combine unsuccessful URLs from both extraction stages
all_unsuccessful_urls = set(unsuccessful_urls).union(set(extraction_unsuccessful_urls))

Processing time for https://www.parkbank.bank/Pages/p-checking.html: 4.15 seconds
Processing time for https://www.parkbank.bank/Pages/savings-cd.html: 41.40 seconds
Total processing time: 45.56 seconds


In [ ]:
print_csv_tables(csv_tables_dict)

In [ ]:
merged_bank_data = merge_json_objects_list(extracted_data)

# Print or save the merged JSON data
json_output = json.dumps(merged_bank_data, indent=4)
print(json_output)